In [ ]:
# | default_exp utils

# Imports

In [ ]:
# | export
import json
import os
import subprocess
from collections import Counter, defaultdict
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import yaml
import threading

# Dataset

In [ ]:
# | export
class Dataset:
    def __init__(
        self,
        data_path: str,
        split: str = "train",
        load_cluster: bool = False,
        return_np_array: bool = True,
    ) -> None:
        """
        Initialize dataset loader.

        Args:
            data_path (str or Path): Base path to the SemanticKITTI dataset.
            split (str): Dataset split to use ('train', 'valid', or 'test').
        """
        self.data_path: Path = Path(data_path)
        self.split: str = split
        self.is_test: bool = split == "test"
        self.load_cluster: bool = load_cluster
        self.return_np_array = return_np_array

        # Paths to YAML config and data folders
        self.yaml_path: Path = Path("../tests/semantic-kitti.yaml")
        # self.yaml_path: Path = self.data_path / 'semantic-kitti.yaml'
        self.velodynes_path: Path = (
            self.data_path / "data_odometry_velodyne/dataset/sequences"
        )
        self.labels_path: Path = (
            self.data_path / "data_odometry_labels/dataset/sequences"
        )
        self.clusters_path: Path = (
            self.data_path / "data_odometry_clusters/dataset/sequences"
        )

        # Load dataset metadata and label mappings
        with open(self.yaml_path, "r") as file:
            metadata: dict = yaml.safe_load(file)

        self.sequences: list[int] = metadata["split"][split]
        self.learning_map: dict[int, int] = metadata["learning_map"]

        # Convert label map to numpy for fast lookup
        max_label: int = max(self.learning_map.keys())
        self.learning_map_np: np.ndarray = np.zeros((max_label + 1,), dtype=np.uint32)
        for raw_label, mapped_label in self.learning_map.items():
            self.learning_map_np[raw_label] = mapped_label

        # Collect all frame paths for selected sequences
        self.frame_paths: list[tuple[str, str]] = self._collect_frame_paths()

        self.last_seq = None
        self.last_frame_id = None

    def _collect_frame_paths(self) -> "list[tuple[str, str]]":
        """Collect all (sequence, frame_id) pairs from the dataset split."""
        frame_list = []
        for seq in self.sequences:
            seq_str = f"{int(seq):02d}"
            seq_velo_path = self.velodynes_path / seq_str / "velodyne"
            velo_files = sorted(seq_velo_path.glob("*.bin"))
            for file in velo_files:
                frame_list.append((seq_str, file.stem))
        return frame_list

    def __len__(self) -> int:
        """Return number of samples in the dataset split."""
        return len(self.frame_paths)

    def _compute_scanline_ids(
        self, point_cloud: np.ndarray, n_scans: int = 64
    ) -> np.ndarray:
        """
        Approximate scanline indices based on point order.

        Args:
            point_cloud (np.ndarray): Nx3 array of 3D points.
            n_scans (int): Number of LiDAR scanlines (e.g., 64 for HDL-64E).

        Returns:
            np.ndarray: Nx1 array with estimated scanline indices (0 to n_scans - 1).
        """
        total_points = point_cloud.shape[0]
        scanline_ids = np.floor(
            np.linspace(0, n_scans, total_points, endpoint=False)
        ).astype(int)
        return scanline_ids.reshape(-1, 1)

    def get_last_seq_frame(self) -> tuple[str, str] | None:
        """
        Get the last sequence and frame ID loaded.

        Returns:
            tuple: (sequence, frame_id)
        """

        if self.last_seq is None or self.last_frame_id is None:
            return None
        else:
            return self.last_seq, self.last_frame_id

    def __getitem__(self, idx: int) -> "dict[str, np.ndarray] | np.ndarray":
        """
        Load a sample from the dataset.

        Args:
            idx (int): Index of the frame to load.

        Returns:
            tuple:
                - point_cloud_with_label (np.ndarray): Nx6 array [x, y, z, true_label, pred_label, scanline_id].
                - item_dict (dict): Contains 'point_cloud', 'label', and 'mask'.
        """

        seq, frame_id = self.frame_paths[idx]
        self.last_seq = seq
        self.last_frame_id = frame_id

        # Load point cloud (Nx4), drop reflectance
        velodyne_file_path = self.velodynes_path / seq / "velodyne" / f"{frame_id}.bin"
        with open(velodyne_file_path, "rb") as file:
            point_cloud = np.fromfile(file, dtype=np.float32).reshape(-1, 4)[:, :3]

        # Load and map semantic labels
        if not self.is_test:
            label_file_path = self.labels_path / seq / "labels" / f"{frame_id}.label"
            if label_file_path.exists():
                with open(label_file_path, "rb") as file:
                    raw_labels = np.fromfile(file, dtype=np.uint32) & 0xFFFF
                labels = self.learning_map_np[raw_labels]
                mask = labels != 0
            else:
                labels = np.zeros(point_cloud.shape[0], dtype=np.uint32)
                mask = np.ones(point_cloud.shape[0], dtype=bool)
        else:
            labels = np.zeros(point_cloud.shape[0], dtype=np.uint32)
            mask = np.ones(point_cloud.shape[0], dtype=bool)

        # Estimate scanline indices
        scanline_ids = self._compute_scanline_ids(point_cloud)

        if self.load_cluster:
            cluster_file_path = (
                self.clusters_path / seq / "clusters" / f"{frame_id}_cluster.label"
            )
            if cluster_file_path.exists():
                with open(cluster_file_path, "rb") as file:
                    clusters = np.fromfile(file, dtype=np.uint32)
                if clusters.shape[0] != point_cloud.shape[0]:
                    raise ValueError(
                        f"Predicted label count mismatch for frame {seq}/{frame_id}"
                    )
                clusters = clusters.reshape(-1, 1)
            else:
                clusters = np.zeros((point_cloud.shape[0], 1), dtype=np.uint32)
        else:
            clusters = np.zeros((point_cloud.shape[0], 1), dtype=np.uint32)

        # # Final format: [x, y, z, true_label, predicted_label, scanline_id]
        point_cloud_array = np.hstack(
            (point_cloud, labels.reshape(-1, 1), clusters, scanline_ids)
        )

        item_dict = {
            "point_cloud": point_cloud,
            "label": labels,
            "cluster": clusters,
            "scanline_id": scanline_ids,
            "mask": mask,
        }

        if self.return_np_array:
            return point_cloud_array
        else:
            return item_dict

# Visualizer for Point Clouds

In [ ]:
# | export
class PointCloudVisualizer:
    def __init__(
        self,
        point_size: float = 1.0,
        grid_size=50,
        grid_spacing=1.0,
        grid_line_width=10,
    ):
        """
        Visualizer class for rendering point clouds using Open3D with color-coded semantic labels.

        Args:
            point_size (float): Default size of points in the Open3D viewer.
        """
        self.point_size = point_size
        self.fixed_colors_rgb = self._get_fixed_colors_rgb()
        self.grid_size = grid_size
        self.grid_spacing = grid_spacing
        self.grid_line_width = grid_line_width

    def set_point_sieze(self, point_size):
        self.point_size = point_size

    def set_grid_size(self, grid_size):
        self.grid_size = grid_size

    def set_grid_spacing(self, grid_spacing):
        self.grid_spacing = grid_spacing

    def set_grid_line_width(self, grid_line_width):
        self.grid_line_width = grid_line_width

    def _get_fixed_colors_rgb(self) -> "dict[int, list[float]]":
        fixed_colors = {
            -1: [255, 255, 255],  # plane
            0: [0, 0, 0],  # unlabeled
            1: [245, 150, 100],  # car
            2: [245, 230, 100],  # bicycle
            3: [150, 60, 30],  # motorcycle
            4: [180, 30, 80],  # truck
            5: [250, 80, 100],  # other-vehicle
            6: [30, 30, 255],  # person
            7: [200, 40, 255],  # bicyclist
            8: [90, 30, 150],  # motorcyclist
            9: [255, 0, 255],  # road
            10: [255, 150, 255],  # parking
            11: [75, 0, 75],  # sidewalk
            12: [75, 0, 175],  # other-ground
            13: [0, 200, 255],  # building
            14: [50, 120, 255],  # fence
            15: [0, 175, 0],  # vegetation
            16: [0, 60, 135],  # trunk
            17: [80, 240, 150],  # terrain
            18: [150, 240, 255],  # pole
            19: [0, 0, 255],  # traffic-sign
        }
        return {
            label: [c / 255.0 for c in reversed(rgb)]
            for label, rgb in fixed_colors.items()
        }

    def _get_color_map(self, labels: np.ndarray) -> np.ndarray:
        """
        Assigns RGB colors to labels.

        Args:
            labels (np.ndarray): Array of label ids.

        Returns:
            np.ndarray: Nx3 array of RGB colors.
        """
        color_map = np.zeros((labels.shape[0], 3))
        for i, label in enumerate(labels):
            if label in self.fixed_colors_rgb:
                color_map[i] = self.fixed_colors_rgb[label]
            else:
                rng = np.random.default_rng(label)
                color_map[i] = rng.random(3)
        return color_map

    def _create_grid(self, grid_size=50, grid_spacing=1.0):
        points = []
        lines = []
        colors = []

        for i in range(-grid_size, grid_size + 1):
            # Linhas paralelas ao eixo X
            points.append([i * grid_spacing, -grid_size * grid_spacing, 0])
            points.append([i * grid_spacing, grid_size * grid_spacing, 0])
            lines.append([len(points) - 2, len(points) - 1])

            # Linhas paralelas ao eixo Y
            points.append([-grid_size * grid_spacing, i * grid_spacing, 0])
            points.append([grid_size * grid_spacing, i * grid_spacing, 0])
            lines.append([len(points) - 2, len(points) - 1])

            # Cores: mais clara a cada 10 unidades
            color = [0.3, 0.3, 0.3] if i % 10 else [0.6, 0.6, 0.6]
            colors.extend([color, color])

        grid = o3d.geometry.LineSet()
        grid.points = o3d.utility.Vector3dVector(points)
        grid.lines = o3d.utility.Vector2iVector(lines)
        grid.colors = o3d.utility.Vector3dVector(colors)
        return grid

    def _create_plane(self, normal_d_tuple, size=100.0):
        """
        Cria um plano baseado no vetor normal e no valor d.

        Args:
            normal_d_tuple (tuple): Tuple contendo o vetor normal (x, y, z) e o valor d.
            size (float): Tamanho do plano a ser desenhado.

        Returns:
            o3d.geometry.TriangleMesh: Mesh do plano.
        """
        normal, d = normal_d_tuple

        # Normalizar o vetor normal
        normal = np.array(normal)
        normal = normal / np.linalg.norm(normal)

        # Calcular 4 pontos do plano
        p1 = np.array(
            [-size, -size, -(normal[0] * (-size) + normal[1] * (-size) + d) / normal[2]]
        )
        p2 = np.array(
            [size, -size, -(normal[0] * size + normal[1] * (-size) + d) / normal[2]]
        )
        p3 = np.array(
            [size, size, -(normal[0] * size + normal[1] * size + d) / normal[2]]
        )
        p4 = np.array(
            [-size, size, -(normal[0] * (-size) + normal[1] * size + d) / normal[2]]
        )

        # Criar os pontos para o mesh
        points = np.vstack((p1, p2, p3, p4))

        # Criar os triângulos que formam o plano
        triangles = [[0, 1, 2], [0, 2, 3]]  # Triângulo 1  # Triângulo 2

        # Criar a malha triangular
        plane_mesh = o3d.geometry.TriangleMesh()
        plane_mesh.vertices = o3d.utility.Vector3dVector(points)
        plane_mesh.triangles = o3d.utility.Vector3iVector(triangles)

        # Colorir a malha do plano de azul
        plane_mesh.paint_uniform_color([0.45, 0.45, 0.45])

        return plane_mesh

    def _create_axis_arrow(self, length=1.0, color=[1, 0, 0], rotation=None):
        arrow = o3d.geometry.TriangleMesh.create_arrow(
            cylinder_radius=0.01,
            cone_radius=0.03,
            cylinder_height=length * 0.8,
            cone_height=length * 0.2,
        )
        arrow.compute_vertex_normals()
        arrow.paint_uniform_color(color)
        if rotation is not None:
            arrow.rotate(rotation, center=(0, 0, 0))
        return arrow

    def show(self, *args, **kwargs):
        thread = threading.Thread(
            target=self.show_point_cloud, args=args, kwargs=kwargs
        )
        thread.start()

    def show_point_cloud(
        self,
        point_cloud: np.ndarray,
        normal_d_tuple: tuple | None = None,
        show_true_label: bool = False,
        show_ground: bool = True,
        show_clusters: bool = True,
        show_unlabeled: bool = True,
        show_plane: bool = False,
        show_grid: bool = False,
    ) -> None:
        """
        Visualize the filtered point cloud using Open3D.

        Args:
            point_cloud (np.ndarray): N x 6 array [x, y, z, true_label, pred_label, scanline_id].
        """

        label_col = 3 if show_true_label else 4
        labels = point_cloud[:, label_col]

        # Apply filter mask
        mask = (
            (show_plane & (labels == -1))
            | (show_unlabeled & (labels == 0))
            | (show_ground & (labels == 9))
            | (show_clusters & (labels >= 1) & (labels != 9))
        )

        xyz = point_cloud[mask, :3]
        visible_labels = labels[mask].astype(int)
        colors = self._get_color_map(visible_labels)

        # Visualize with point size
        vis = o3d.visualization.Visualizer()  # type: ignore
        vis.create_window(window_name="Plane Visualization", width=800, height=600)

        opt = vis.get_render_option()
        opt.point_size = self.point_size
        opt.background_color = np.asarray([0.1, 0.1, 0.1])  # estilo AutoCAD / pptk
        opt.show_coordinate_frame = True
        opt.mesh_show_back_face = True  # Exibir o lado de trás do plano

        # extrair para uma funcao
        # X (vermelho): rotaciona -90° ao redor Z
        arrow_x = self._create_axis_arrow(
            length=1.0,
            color=[1, 0, 0],
            rotation=o3d.geometry.get_rotation_matrix_from_xyz([0, np.pi / 2, 0]),
        )
        # Y (verde): rotaciona +90° ao redor X
        arrow_y = self._create_axis_arrow(
            length=1.0,
            color=[0, 1, 0],
            rotation=o3d.geometry.get_rotation_matrix_from_xyz([-np.pi / 2, 0, 0]),
        )
        # Z (azul): já está na direção Z por padrão
        arrow_z = self._create_axis_arrow(length=1.0, color=[0, 0, 1], rotation=None)
        vis.add_geometry(arrow_x)
        vis.add_geometry(arrow_y)
        vis.add_geometry(arrow_z)

        # Create Open3D point cloud
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(xyz)
        pcd.colors = o3d.utility.Vector3dVector(colors)
        vis.add_geometry(pcd)

        if show_grid:
            vis.add_geometry(
                self._create_grid(self.grid_size, self.grid_spacing)
            )  # grid grande com espaçamento 1m
            opt.line_width = self.grid_line_width
        if show_plane and normal_d_tuple != None:
            vis.add_geometry(self._create_plane(normal_d_tuple))

        vis.run()
        vis.destroy_window()

In [ ]:
# | export
def rodar_visualizador(point_cloud, normal_d_tuple=None, **visualizer_params):
    temp_dir = "temp_vis"
    os.makedirs(temp_dir, exist_ok=True)

    # Salvar nuvem de pontos
    np.save(os.path.join(temp_dir, "point_cloud.npy"), point_cloud)

    # Salvar plano (normal + d)
    if normal_d_tuple:
        normal_d = np.array([*normal_d_tuple[0], normal_d_tuple[1]])
        np.save(os.path.join(temp_dir, "normal_d.npy"), normal_d)

    # Salvar parâmetros
    with open(os.path.join(temp_dir, "visualizer_config.json"), "w") as f:
        json.dump(visualizer_params, f)

    # Rodar visualizador em subprocesso
    comando = ["python3", "visualizer.py", temp_dir]
    thread = threading.Thread(target=lambda: subprocess.run(comando))
    thread.start()

# Save

In [ ]:
# | export
def save_clusters(
    point_cloud: np.ndarray, seq: str, frame_id: str, output_base_path: str
) -> None:
    """
    Salva os rótulos preditos em formato .label seguindo a estrutura do SemanticKITTI.

    Args:
        point_cloud (np.ndarray): Array Nx6 com colunas [x, y, z, true_label, predicted_label, scanline_index].
        seq (str): Número da sequência (ex: '00', '01', ...).
        frame_id (str): ID do frame (ex: '000123').
        output_base_path (str): Caminho base até `data_odometry_cluster_pred/dataset/sequences`.
    """
    # Extrair coluna de predição (índice 4)
    predicted_labels = point_cloud[:, 4].astype(np.uint32)

    # Construir caminho de saída
    output_dir = Path(output_base_path) / seq / "clusters"
    output_dir.mkdir(parents=True, exist_ok=True)

    # Nome do arquivo com sufixo _pred.label
    output_file = output_dir / f"{frame_id}_cluster.label"

    # Salvar como arquivo binário .label
    predicted_labels.tofile(output_file)

# Result analisys

In [ ]:
# | export
def verificar_consistencia_labels(pontos):
    """
    Verifica inconsistências entre rótulos verdadeiros e clusters atribuídos.
    Também calcula estatísticas (média e desvio padrão) dos clusters inconsistentes.

    Retorna:
        - inconsistentes_total (int): número de clusters inconsistentes
        - combinacoes_contadas (Counter): contagem de combinações de labels inconsistentes
        - estatisticas_erro (dict): médias e desvios padrão das métricas de erro
    """
    labels_pred = pontos[:, 4].astype(int)
    labels_true = pontos[:, 3].astype(int)

    pares = np.stack((labels_pred, labels_true), axis=1)

    cluster_to_true_labels = defaultdict(list)
    for cluster_id, true_label in pares:
        cluster_to_true_labels[cluster_id].append(true_label)

    combinacoes = []
    inconsistentes_total = 0

    # Para calcular as médias e desvios depois
    lista_total_pontos = []
    lista_pontos_label_dominante = []
    lista_erro_percentual = []

    for cluster_id, labels in cluster_to_true_labels.items():
        if cluster_id == 9:
            continue  # Ignorar o solo

        labels_sem_unlabeled = [l for l in labels if l != 0]

        if len(set(labels_sem_unlabeled)) > 1:
            inconsistentes_total += 1

            # Cria combinação ordenada para agrupar
            combinacao = tuple(sorted(set(labels_sem_unlabeled)))
            combinacoes.append(combinacao)

            # Estatísticas de erro
            total_pontos = len(labels)
            contador_labels = Counter(labels_sem_unlabeled)
            label_dominante, pontos_label_dominante = contador_labels.most_common(1)[0]
            erro_percentual = 100 * (1 - (pontos_label_dominante / total_pontos))

            lista_total_pontos.append(total_pontos)
            lista_pontos_label_dominante.append(pontos_label_dominante)
            lista_erro_percentual.append(erro_percentual)

    combinacoes_contadas = Counter(combinacoes)

    # Se não houver inconsistentes, evita erro de divisão
    if inconsistentes_total > 0:
        estatisticas_erro = {
            "media_total_pontos": np.mean(lista_total_pontos),
            "std_total_pontos": np.std(lista_total_pontos),
            "media_pontos_label_dominante": np.mean(lista_pontos_label_dominante),
            "std_pontos_label_dominante": np.std(lista_pontos_label_dominante),
            "media_erro_percentual": np.mean(lista_erro_percentual),
            "std_erro_percentual": np.std(lista_erro_percentual),
        }
    else:
        estatisticas_erro = {
            "media_total_pontos": 0,
            "std_total_pontos": 0,
            "media_pontos_label_dominante": 0,
            "std_pontos_label_dominante": 0,
            "media_erro_percentual": 0,
            "std_erro_percentual": 0,
        }

    return inconsistentes_total, combinacoes_contadas, estatisticas_erro

In [ ]:
# | export
def get_statistics(idx, point_cloud):
    clusters = point_cloud[:, 4]
    num_clusters = len(np.unique(clusters))
    num_pontos = point_cloud.shape[0]

    cluster_sizes = [np.sum(clusters == label) for label in np.unique(clusters)]
    cluster_sizes_sorted = sorted(cluster_sizes, reverse=True)

    ground_cluster_size = np.sum(clusters == 9)
    largest_non_ground_cluster = (
        cluster_sizes_sorted[1]
        if cluster_sizes_sorted[0] == ground_cluster_size
        and len(cluster_sizes_sorted) > 1
        else cluster_sizes_sorted[0]
    )

    num_clusters_inconsistentes, combinacoes, erros_clusters = (
        verificar_consistencia_labels(point_cloud)
    )

    estatisticas_do_frame = {
        "frame_id": idx,
        "num_clusters": num_clusters,
        "num_pontos": num_pontos,
        "pontos_por_cluster_medio": (
            num_pontos / num_clusters if num_clusters > 0 else 0
        ),
        "largest_non_ground_cluster": largest_non_ground_cluster,
        "gound_cluster_size": ground_cluster_size,
        "num_clusters_inconsistentes": num_clusters_inconsistentes,  # Novo campo
        "combinacoes": combinacoes,
        "erros_clusters": erros_clusters,
    }

    return estatisticas_do_frame

# Plots

In [ ]:
# | export
def plot_1(resumo_por_frame, seq_value=None):
    """
    Plota em 4 subplots verticais:
    1. Número total de pontos
    2. Número total de clusters
    3. Pontos por cluster médio
    4. Porcentagem de redução da dimensionalidade
    """

    # Extrair dados
    num_clusters = np.array([f["num_clusters"] for f in resumo_por_frame])
    num_pontos = np.array([f["num_pontos"] for f in resumo_por_frame])
    pontos_por_cluster_medio = np.array(
        [f["pontos_por_cluster_medio"] for f in resumo_por_frame]
    )
    frames = np.arange(len(resumo_por_frame))

    # Calcular a redução de dimensionalidade
    reducao_dimensionalidade = 1 - (num_clusters / num_pontos)

    # Criar subplots
    fig, axs = plt.subplots(4, 1, figsize=(12, 18))  # 4 linhas, 1 coluna
    fig.tight_layout(pad=5.0)

    # Plot 1 - Número de pontos
    media_pontos = np.mean(num_pontos)
    desvio_pontos = np.std(num_pontos)
    axs[0].plot(frames, num_pontos, marker="o", label="Número de Pontos", color="green")
    axs[0].axhline(
        media_pontos, color="red", linestyle="--", label=f"Média = {media_pontos:.1f}"
    )
    axs[0].fill_between(
        frames,
        media_pontos - desvio_pontos,
        media_pontos + desvio_pontos,
        color="red",
        alpha=0.2,
        label=f"Desvio = {desvio_pontos:.1f}",
    )
    axs[0].set_title("Número de Pontos por Frame")
    axs[0].set_xlabel("Frame")
    axs[0].set_ylabel("Número de Pontos")
    axs[0].legend()
    axs[0].grid(True)

    # Plot 2 - Número de clusters
    media_clusters = np.mean(num_clusters)
    desvio_clusters = np.std(num_clusters)
    axs[1].plot(
        frames, num_clusters, marker="o", label="Número de Clusters", color="blue"
    )
    axs[1].axhline(
        media_clusters,
        color="blue",
        linestyle="--",
        label=f"Média = {media_clusters:.1f}",
    )
    axs[1].fill_between(
        frames,
        media_clusters - desvio_clusters,
        media_clusters + desvio_clusters,
        color="blue",
        alpha=0.2,
        label=f"Desvio = {desvio_clusters:.1f}",
    )
    axs[1].set_title("Número de Clusters por Frame")
    axs[1].set_xlabel("Frame")
    axs[1].set_ylabel("Número de Clusters")
    axs[1].legend()
    axs[1].grid(True)

    # Plot 3 - Pontos por cluster médio
    media_ppc = np.mean(pontos_por_cluster_medio)
    desvio_ppc = np.std(pontos_por_cluster_medio)
    axs[2].plot(
        frames,
        pontos_por_cluster_medio,
        marker="o",
        label="Pontos por Cluster Médio",
        color="orange",
    )
    axs[2].axhline(
        media_ppc, color="orange", linestyle="--", label=f"Média = {media_ppc:.1f}"
    )
    axs[2].fill_between(
        frames,
        media_ppc - desvio_ppc,
        media_ppc + desvio_ppc,
        color="orange",
        alpha=0.2,
        label=f"Desvio = {desvio_ppc:.1f}",
    )
    axs[2].set_title("Média de Pontos por Cluster por Frame")
    axs[2].set_xlabel("Frame")
    axs[2].set_ylabel("Pontos por Cluster")
    axs[2].legend()
    axs[2].grid(True)

    # Plot 4 - Redução da dimensionalidade
    media_reducao = np.mean(reducao_dimensionalidade)
    desvio_reducao = np.std(reducao_dimensionalidade)
    axs[3].plot(
        frames,
        reducao_dimensionalidade * 100,
        marker="o",
        label="Redução Dimensionalidade (%)",
        color="purple",
    )
    axs[3].axhline(
        media_reducao * 100,
        color="purple",
        linestyle="--",
        label=f"Média = {media_reducao*100:.2f}%",
    )
    axs[3].fill_between(
        frames,
        (media_reducao - desvio_reducao) * 100,
        (media_reducao + desvio_reducao) * 100,
        color="purple",
        alpha=0.2,
        label=f"Desvio = {desvio_reducao*100:.2f}%",
    )
    axs[3].set_title("Porcentagem de Redução da Dimensionalidade por Frame")
    axs[3].set_xlabel("Frame")
    axs[3].set_ylabel("Redução (%)")
    axs[3].legend()
    axs[3].grid(True)

    # Ajuste layout final
    plt.tight_layout()
    plt.show()
    if seq_value is not None:
        Path("plots").mkdir(parents=True, exist_ok=True)
        Path(f"plots/{seq_value}").mkdir(parents=True, exist_ok=True)
        # Salvar o gráfico
        plt.savefig(f"plots/{seq_value}/plot-1.png", dpi=300, bbox_inches="tight")

In [ ]:
# | export
def plot_2(resumo_por_frame, seq_value=None):
    gound_cluster_size = [f["gound_cluster_size"] for f in resumo_por_frame]
    largest_non_ground_cluster = [
        f["largest_non_ground_cluster"] for f in resumo_por_frame
    ]

    def get_stats(values):
        return np.mean(values), np.std(values)

    mean_9, std_9 = get_stats(gound_cluster_size)
    mean_second, std_second = get_stats(largest_non_ground_cluster)

    fig, axs = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

    # Cluster 9
    axs[0].plot(gound_cluster_size, marker="o", label="Ground cluster")
    axs[0].axhline(mean_9, color="green", linestyle="--", label=f"Média = {mean_9:.1f}")
    axs[0].fill_between(
        range(len(gound_cluster_size)),
        mean_9 - std_9,
        mean_9 + std_9,
        color="green",
        alpha=0.2,
        label=f"Desvio = {std_9:.1f}",
    )
    axs[0].set_ylabel("Tamanho")
    axs[0].set_title("Tamanho do Ground Cluster")
    axs[0].legend()
    axs[0].grid(True)

    # Segundo maior
    axs[1].plot(
        largest_non_ground_cluster, marker="o", label="Maior non ground cluster"
    )
    axs[1].axhline(
        mean_second, color="blue", linestyle="--", label=f"Média = {mean_second:.1f}"
    )
    axs[1].fill_between(
        range(len(largest_non_ground_cluster)),
        mean_second - std_second,
        mean_second + std_second,
        color="blue",
        alpha=0.2,
        label=f"Desvio = {std_second:.1f}",
    )
    axs[1].set_ylabel("Tamanho")
    axs[1].set_title("Tamanho do Maior non Ground Cluster")
    axs[1].legend()
    axs[1].grid(True)

    plt.tight_layout()
    plt.show()
    if seq_value is not None:
        Path("plots").mkdir(parents=True, exist_ok=True)
        Path(f"plots/{seq_value}").mkdir(parents=True, exist_ok=True)
        # Salvar o gráfico
        fig.savefig(f"plots/{seq_value}/plot-2.png", dpi=300, bbox_inches="tight")

In [ ]:
# | export
def plot_3(resumo_por_frame, seq_value=None):
    """
    Plota a quantidade de clusters inconsistentes por frame,
    incluindo média e desvio padrão, juntamente com a porcentagem de erro.
    """
    inconsistentes_por_frame = np.array(
        [frame["num_clusters_inconsistentes"] for frame in resumo_por_frame]
    )
    num_clusters_por_frame = np.array(
        [frame["num_clusters"] for frame in resumo_por_frame]
    )

    # Calcular a porcentagem de erro
    porcentagem_erro = 1 - (inconsistentes_por_frame / num_clusters_por_frame)
    porcentagem_erro = np.clip(porcentagem_erro, 0, 1)

    # Média e desvio padrão de inconsistências
    media_inconsistencias = np.mean(inconsistentes_por_frame)
    desvio_inconsistencias = np.std(inconsistentes_por_frame)

    # Média e desvio padrão de porcentagem de erro
    media_erro = np.mean(porcentagem_erro)
    desvio_erro = np.std(porcentagem_erro)

    # --- Aqui a mudança ---
    fig, axs = plt.subplots(2, 1, figsize=(10, 10), sharex=True)

    # Gráfico de inconsistências
    axs[0].plot(inconsistentes_por_frame, marker="o", label="Inconsistências por frame")
    axs[0].axhline(
        media_inconsistencias,
        color="red",
        linestyle="--",
        label=f"Média = {media_inconsistencias:.2f}",
    )
    axs[0].fill_between(
        range(len(inconsistentes_por_frame)),
        media_inconsistencias - desvio_inconsistencias,
        media_inconsistencias + desvio_inconsistencias,
        color="red",
        alpha=0.2,
        label=f"Desvio padrão = {desvio_inconsistencias:.2f}",
    )
    axs[0].set_title("Clusters Inconsistentes por Frame")
    axs[0].set_xlabel("Frame")
    axs[0].set_ylabel("Nº de Clusters Inconsistentes")
    axs[0].grid(True)
    axs[0].legend()

    # Gráfico de porcentagem de erro
    axs[1].plot(
        porcentagem_erro,
        marker="o",
        label="Porcentagem de acerto dado quantidade de clusters",
    )
    axs[1].axhline(
        media_erro, color="green", linestyle="--", label=f"Média = {media_erro:.2f}"
    )
    axs[1].fill_between(
        range(len(porcentagem_erro)),
        media_erro - desvio_erro,
        media_erro + desvio_erro,
        color="green",
        alpha=0.2,
        label=f"Desvio padrão = {desvio_erro:.2f}",
    )
    axs[1].set_title("Porcentagem de Erro por Frame")
    axs[1].set_xlabel("Frame")
    axs[1].set_ylabel("Porcentagem de Erro")
    axs[1].grid(True)
    axs[1].legend()

    plt.tight_layout()
    plt.show()
    # Salvar antes de mostrar
    if seq_value is not None:
        Path("plots").mkdir(parents=True, exist_ok=True)
        Path(f"plots/{seq_value}").mkdir(parents=True, exist_ok=True)
        # Salvar o gráfico
        fig.savefig(f"plots/{seq_value}/plot-3.png", dpi=300, bbox_inches="tight")

In [ ]:
# | export
def plot_4(resumo_por_frame, seq_value=None):
    """
    Plota três métricas de erro em subplots separados, mas no mesmo figure.
    """

    # Configurações para as 3 métricas
    metricas = [
        {
            "chave_media": "media_total_pontos",
            "chave_std": "std_total_pontos",
            "titulo": "Total de Pontos nos Clusters Inconsistentes por Frame",
            "ylabel": "Total de Pontos",
        },
        {
            "chave_media": "media_pontos_label_dominante",
            "chave_std": "std_pontos_label_dominante",
            "titulo": "Pontos do Label Dominante por Cluster Inconsistente",
            "ylabel": "Pontos do Label Dominante",
        },
        {
            "chave_media": "media_erro_percentual",
            "chave_std": "std_erro_percentual",
            "titulo": "Erro Percentual nos Clusters Inconsistentes por Frame",
            "ylabel": "Erro Percentual (%)",
        },
    ]

    # Criar subplots
    fig, axs = plt.subplots(3, 1, figsize=(12, 15))  # 3 linhas, 1 coluna
    fig.tight_layout(pad=5.0)

    for idx, config in enumerate(metricas):
        chave_media = config["chave_media"]
        chave_std = config["chave_std"]
        titulo = config["titulo"]
        ylabel = config["ylabel"]

        metricas_por_frame = np.array(
            [frame["erros_clusters"][chave_media] for frame in resumo_por_frame]
        )
        media_global = np.mean(metricas_por_frame)
        desvio_global = np.std(metricas_por_frame)

        ax = axs[idx]
        ax.plot(metricas_por_frame, marker="o", label=f"{ylabel} por frame")
        ax.axhline(
            media_global,
            color="red",
            linestyle="--",
            label=f"Média = {media_global:.2f}",
        )
        ax.fill_between(
            range(len(metricas_por_frame)),
            media_global - desvio_global,
            media_global + desvio_global,
            color="red",
            alpha=0.2,
            label=f"Desvio padrão = {desvio_global:.2f}",
        )
        ax.set_title(titulo)
        ax.set_xlabel("Frame")
        ax.set_ylabel(ylabel)
        ax.legend()
        ax.grid(True)

    plt.show()
    if seq_value is not None:
        Path("plots").mkdir(parents=True, exist_ok=True)
        Path(f"plots/{seq_value}").mkdir(parents=True, exist_ok=True)
        # Salvar o gráfico
        fig.savefig(f"plots/{seq_value}/plot-4.png", dpi=300, bbox_inches="tight")

In [ ]:
# | export
def plot_5(combinacoes_geral, seq_value=None, top_n=15):
    label_id_to_name = {
        -1: "plane",
        0: "unlabeled",
        1: "car",
        2: "bicycle",
        3: "motorcycle",
        4: "truck",
        5: "other-vehicle",
        6: "person",
        7: "bicyclist",
        8: "motorcyclist",
        9: "road",
        10: "parking",
        11: "sidewalk",
        12: "other-ground",
        13: "building",
        14: "fence",
        15: "vegetation",
        16: "trunk",
        17: "terrain",
        18: "pole",
        19: "traffic-sign",
    }

    # total_inconsistentes = sum(combinacoes_geral.values())

    # Ordena as combinações do mais frequente para o menos frequente

    # for combinacao, count in combinacoes_ordenadas:
    #     nomes_labels = [label_id_to_name.get(label, f"desconhecido({label})") for label in combinacao]
    #     nomes_labels_str = ", ".join(nomes_labels)

    #     porcentagem = (count / total_inconsistentes) * 100

    combinacoes_ordenadas = sorted(
        combinacoes_geral.items(), key=lambda x: x[1], reverse=True
    )
    # Extrair as combinações e suas contagens
    top_combinacoes = combinacoes_ordenadas[:top_n]

    # Separar nomes e valores
    def formatar_combinacao(combinacao):
        nomes = [label_id_to_name.get(int(l), f"desconhecido({l})") for l in combinacao]
        return ", ".join(nomes)

    labels_pizza = [formatar_combinacao(comb) for comb, _ in top_combinacoes]
    valores_pizza = [contagem for _, contagem in top_combinacoes]

    # Calcular "outros"
    total = sum(combinacoes_geral.values())
    total_top = sum(valores_pizza)
    outros = total - total_top

    if outros > 0:
        labels_pizza.append("Outros")
        valores_pizza.append(outros)

    fig, ax = plt.subplots(figsize=(12, 10))
    ax.pie(valores_pizza, labels=labels_pizza, autopct="%1.1f%%", startangle=140)
    ax.set_title(f"Top {top_n} combinações inconsistentes + Outros")
    ax.axis("equal")
    plt.tight_layout()
    plt.show()
    if seq_value is not None:
        Path("plots").mkdir(parents=True, exist_ok=True)
        Path(f"plots/{seq_value}").mkdir(parents=True, exist_ok=True)
        # Salvar o gráfico
        fig.savefig(f"plots/{seq_value}/plot-5.png", dpi=300, bbox_inches="tight")